## Notebook to show how to use redback to generate toO style observations for any model. 

You will need to install Redback. Instructions available at https://redback.readthedocs.io/en/latest/. I suggest installing from source via GitHub.

In [ ]:
import redback
import pandas as pd
from redback.simulate_transients import SimulateOpticalTransient
import matplotlib.pyplot as plt
import numpy as np

We first design a strategy. This takes the form of a dataframe which specifies sky pointings, time, limiting mags, and the bands

In [ ]:
# specify the number of pointings per band 
num_obs = {'lsstg': 10, 'lsstr':10, 'lssti':10, 'lsstz':10, 'lsstu':10}

# specify the cadence in days for each band
average_cadence = {'lsstg': 1.5, 'lsstr': 5.0, 'lssti': 2.5, 'lsstz':1, 'lsstu':1}

# specify any scatter on the cadence, the time of the observation will be 
# taken from a Gaussian with the scatter as sigma
cadence_scatter = {'lsstg': 0.5, 'lsstr':0.5, 'lssti':0.5, 'lsstz':1, 'lsstu':1}

# Specify limiting 5 sigma depth magnitude
limiting_magnitudes = {'lsstg': 25.0, 'lsstr': 24.5, 'lssti': 23.0, 'lsstu':25, 'lsstz':23}

# We now use redback to make a pointings table from the above information
# We set RA and DEC to always be at the location of the transient 
# but we can change this to incorporate the fov/full survey
ra = 1.0 
dec = 1.5
# We also set the start time of the observation/survey strategy 
initMJD = 59581.0
pointings = redback.simulate_transients.make_pointing_table_from_average_cadence(
    ra=ra, dec=dec, num_obs=num_obs, average_cadence=average_cadence,
    cadence_scatter=cadence_scatter, limiting_magnitudes=limiting_magnitudes, 
    initMJD=59581.0)
print(pointings)

The pointings are just a dataframe with the specific columns shown above. You could just as easily make your own up or read a table from somewhere else. The above is just meant to be a helpful function to make these pointings

We now specify a redback model (or a user implemented model) and the parameters we want to simulate given the above cadences. 

In [ ]:
model_kwargs = {}
# Any redback model can be referred to as a string. 
# If the user has their own model, they can pass a function here instead. 
# There are over a 100 models implemented in redback, lots of models for kilonovae, GRB afterglows, 
# supernovae, TDEs and other things
model = 'one_component_kilonova'
# Load the default prior for this model in redback and sample from it to get 1 set of parameters. 
# We can sample from the default prior for this model for a random kilonova. 
parameters = redback.priors.get_priors(model=model).sample()

# We fix a few parameters here to create a nice looking kilonova. 
# You can change any of the parameters here or add additional keyword arguments 
# to change some physical assumptions. Please refer to the documentation for this and units etc
parameters['mej'] = 0.05
parameters['t0_mjd_transient'] = 59582.0
parameters['redshift'] = 0.075
parameters['t0'] = parameters['t0_mjd_transient']
parameters['temperature_floor'] = 3000
parameters['kappa'] = 1
parameters['vej'] = 0.2
parameters['ra'] = 1.0
parameters['dec'] = 1.5
print(parameters)

We now simulate a kilonova with the above parameters and the strategy designed above.

In [ ]:
# Specify some additional settings. 
# A threshold for data points to consider detections based on a SNR. 
snr_threshold = 5.0

# A maximum time to evaluate the transient, 
# this is useful if you do not trust the model past a certain time or do not want to generate detections past this time. 
end_transient_time = 20

# Simulate by passing in the parameters of the model, the model string (or a python function), 
# and the pointings generated above.
kn_sim = SimulateOpticalTransient.simulate_transient(model='one_component_kilonova_model',
                                       parameters=parameters, pointings_database=pointings,
                                       survey=None, model_kwargs=model_kwargs,
                                        end_transient_time=20., snr_threshold=5.0)

In [ ]:
# We can print the observations that were simulated to see what the data looks like. 
# This will include extra stuff like non-detections etc
print(kn_sim.observations)

In [ ]:
# We can also save the observations to a file using the save_transient method.
# This will save the observations to a csv file in a 'simulated' directory alongside the csv file
# specifying the injection parameters.
kn_sim.save_transient(name='my_kilonova')

In [ ]:
kn_object = redback.transient.Kilonova.from_simulated_optical_data(name='my_kilonova', data_mode='magnitude')

# Make a dictionary for colors on the plot
band_colors = {'lsstg':'#4daf4a', 'lsstu':'#377eb8', 'lsstr':'#e41a1c', 
               'lsstz':'#a65628', 'lssti':'#ff7f00'}
ax = kn_object.plot_data(show=False, band_colors=band_colors)
# ax.set_ylim(22, 19)

The above plot only shows the detection and not the input lightcurve or non-detections. Let's add those in. As the axes is returned with can use the attributes stored in the kn_sim object directly.

In [ ]:
# Make a dictionary for colors on the plot
band_colors = {'lsstg':'#4daf4a', 'lsstu':'#377eb8', 'lsstr':'#e41a1c', 
               'lsstz':'#a65628', 'lssti':'#ff7f00'}
ax = kn_object.plot_data(show=False, band_colors=band_colors)
ax.set_ylim(28, 22)
upper_limits = kn_sim.observations[kn_sim.observations['detected'] != 1.0]
data = kn_sim.observations[kn_sim.observations['detected'] == 1.0]
for band in band_colors.keys():
    up = upper_limits[upper_limits['band'] == band]
    dd = data[data['band'] == band]
    plt.scatter(dd['time (days)'], dd['magnitude'], s=100, marker='.', color=band_colors[band])
    plt.scatter(up['time (days)'], up['limiting_magnitude'], s=100, marker=r'$\downarrow$', color=band_colors[band])

    
# We can also plot the true data 
tt = np.linspace(0.1, 20, 100)
# specify output_format 
parameters['output_format'] = 'magnitude'
for band in band_colors.keys():
    parameters['bands'] = band
    out = redback.transient_models.kilonova_models.one_component_kilonova_model(tt, **parameters)
    plt.plot(tt, out, color=band_colors[band], alpha=0.3)

plt.xlim(0.1, 10)


You can now use the simulated object and do parameter estimation. There are multiple examples available at 
https://github.com/nikhil-sarin/redback/tree/master/examples. Alongside other examples to simulate full survey or single lightcurves for Rubin or ZTF 
https://github.com/nikhil-sarin/redback/blob/master/examples/simulate_survey.py
https://github.com/nikhil-sarin/redback/blob/master/examples/simulate_single_transient_in_rubin.py
